In [2]:
import pandas as pd
import numpy as np
import datetime 
import math
import matplotlib.pyplot as plt



In [3]:
cs_df = pd.read_excel(io = r'online_retail.xlsx')
print cs_df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object


In [4]:
cs_df.Country.value_counts().head(10)


United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: Country, dtype: int64

In [5]:
cat_des_df = cs_df.groupby(["StockCode","Description"]).count().reset_index()
cat_des_df.StockCode.value_counts()[cat_des_df.StockCode.value_counts()>1].reset_index().head()

,index,StockCode
0,20713,8
1,23084,7
2,21830,6
3,85175,6
4,23131,5


In [6]:
cs_df[cs_df['StockCode']==cat_des_df.StockCode.value_counts()[cat_des_df.StockCode.value_counts()>1].reset_index()['index'][6]]['Description'].unique()

array([u'SET/3 ROSE CANDLE IN JEWELLED BOX', u'wet pallet', u'damages',
       u'???missing', u'AMAZON'], dtype=object)

In [7]:
cs_df = cs_df[cs_df.Country == 'United Kingdom']
cs_df['amount'] = cs_df.Quantity*cs_df.UnitPrice
cs_df = cs_df[~(cs_df.amount<0)]
cs_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [8]:
cs_df = cs_df[~(cs_df.CustomerID.isnull())]
cs_df.shape

(354345, 9)

In [9]:
refrence_date = cs_df.InvoiceDate.max()
refrence_date = refrence_date + datetime.timedelta(days = 1)
refrence_date

Timestamp('2011-12-10 12:49:00')

In [10]:
cs_df['days_since_last_purchase'] = refrence_date - cs_df.InvoiceDate
cs_df['days_since_last_purchase_num'] = cs_df['days_since_last_purchase'].astype('timedelta64[D]')

In [11]:
customer_history_df = cs_df.groupby("CustomerID").min().reset_index()[['CustomerID', 'days_since_last_purchase_num']]

customer_history_df.rename(columns={'days_since_last_purchase_num':'recency'},inplace=True)


In [12]:
customer_monetary_val = cs_df[['CustomerID','amount']].groupby("CustomerID").sum().reset_index()
customer_history_df = customer_history_df.merge(customer_monetary_val, how='outer')
customer_history_df.amount = customer_history_df.amount+0.0001
customer_freq = cs_df[['CustomerID','amount']].groupby("CustomerID").count().reset_index()
customer_freq.rename(columns={'amount':'frequency'},inplace=True)
customer_history_df = customer_history_df.merge(customer_freq, how='outer')

In [13]:
from sklearn import preprocessing
import math
customer_history_df['recency_log'] = customer_history_df['recency'].apply(math.log)
customer_history_df['frequency_log'] = customer_history_df['frequency'].apply(math.log)
customer_history_df['amount_log'] = customer_history_df['amount'].apply(math.log)
feature_vector = ['amount_log', 'recency_log','frequency_log']

In [14]:
X = customer_history_df[feature_vector].as_matrix()
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [15]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

X = X_scaled

cluster_centers = dict()

In [17]:
for n_clusters in range(3,6,2):
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    cluster_centers.update({n_clusters :{
                                        'cluster_center':clusterer.cluster_centers_,
                                        'silhouette_score':silhouette_avg,
                                        'labels':cluster_labels}
                            })
    
    
print cluster_centers

{3: {'cluster_center': array([[ 1.19488018, -1.23262163,  1.13312405],
       [-0.89904048,  0.72804253, -0.96903891],
       [ 0.14668453,  0.01490426,  0.23619168]]), 'silhouette_score': 0.3036592333727506, 'labels': array([2, 0, 0, ..., 1, 0, 2], dtype=int32)}, 5: {'cluster_center': array([[ 1.34447256, -1.42605922,  1.26344122],
       [-0.40363857,  0.82662812, -0.30921621],
       [-0.3352104 , -0.77739446, -0.23043138],
       [-1.24626588,  0.76597928, -1.50627264],
       [ 0.61225789,  0.0492479 ,  0.64897696]]), 'silhouette_score': 0.27961380275479647, 'labels': array([4, 0, 0, ..., 2, 0, 4], dtype=int32)}}
